# AutoEncoder in TensorFlow
- [Reference Link](https://jmetzen.github.io/2015-11-27/vae.html) by Jan Hendrik Metzen
- [Autoencoder_models](https://github.com/tensorflow/models/tree/master/autoencoder/autoencoder_models)
- [DenoisingAutoencoder](https://github.com/tensorflow/models/blob/master/autoencoder/autoencoder_models/DenoisingAutoencoder.py)

In [1]:
import numpy as np
import tensorflow as tf
import sklearn.preprocessing as prep
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(0)
tf.set_random_seed(0)

## Xavier initialization

In [2]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
#     W = tf.get_variable("W", shape=[784, 256],
#            initializer=tf.contrib.layers.xavier_initializer())
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)

## Denoising Autoencoder

In [3]:
class AdditiveGaussianNoiseAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.softplus, optimizer = tf.train.AdamOptimizer(),
                 scale = 0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
         # model
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
         # self.x + scale * tf.random_normal((n_input,) <- add noise into input
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + scale * tf.random_normal((n_input,)),
                self.weights['w1']), self.weights['b1']))
        # reconstruct hidden layer output to fit input x
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])

        # cost
        # use MSE 
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
        
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.get_variable("w1", shape=[self.n_input, self.n_hidden],
            initializer=tf.contrib.layers.xavier_initializer())
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype = tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype = tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype = tf.float32))
        return all_weights
    
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict = {self.x: X,
                                                                            self.scale: self.training_scale
                                                                            })
        return cost

    def calc_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict = {self.x: X,
                                                     self.scale: self.training_scale
                                                     })

    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict = {self.x: X,
                                                       self.scale: self.training_scale
                                                       })

    def generate(self, hidden=None):
        if hidden is None:
            hidden = self.sess.run(tf.random_normal([1, self.n_hidden]))
        return self.sess.run(self.reconstruction, feed_dict = {self.hidden: hidden})

    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict = {self.x: X,
                                                               self.scale: self.training_scale
                                                               })

    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])

# Solve MINST Problem Use Autoencoder

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

In [10]:
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]

In [7]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
n_samples = int(mnist.train.num_examples)
training_epochs = 20
batch_size = 128
display_step = 1

In [8]:
autoencoder = AdditiveGaussianNoiseAutoencoder(n_input = 784, n_hidden = 200, 
                                               transfer_function = tf.nn.softplus, 
                                               optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), 
                                               scale = 0.01)


In [12]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
    if epoch % display_step == 0:
        print("Epoch:", '%04d' %(epoch + 1), "cost = ", "{:.9f}".format(avg_cost))

Epoch: 0001 cost =  7800.771584659
Epoch: 0002 cost =  8051.504724432
Epoch: 0003 cost =  9032.765611932
Epoch: 0004 cost =  8687.430390909
Epoch: 0005 cost =  8479.463310795
Epoch: 0006 cost =  7485.702743182
Epoch: 0007 cost =  7436.232067045
Epoch: 0008 cost =  8278.548241477
Epoch: 0009 cost =  7416.342011932
Epoch: 0010 cost =  8041.344284091
Epoch: 0011 cost =  7907.777606250
Epoch: 0012 cost =  7710.017519886
Epoch: 0013 cost =  8018.053495455
Epoch: 0014 cost =  7331.605735227
Epoch: 0015 cost =  8434.755108523
Epoch: 0016 cost =  7980.554456250
Epoch: 0017 cost =  7235.336110227
Epoch: 0018 cost =  8030.771870455
Epoch: 0019 cost =  8083.847779545
Epoch: 0020 cost =  8360.222562500


In [13]:
print("Total cost: " + str(autoencoder.calc_total_cost(X_test)))

Total cost:650100.0
